In [1]:
%run data_setup

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

path_count = 5

# Machine Learning Models

In [3]:
# set aggregate climate risk as a variable

tmp = df.query("path_ind == 0")
tmp_SADC_ij = np.empty(len(tmp))
for t in range(len(years)):
    tmp_yr = tmp.query("Year == {}".format(years[t]))
    for i in range(len(tmp_yr)):
        intervening_states = states[distance[tmp_yr.iloc[i]['State_i']] < distance[tmp_yr.iloc[i]['State_i']][tmp_yr.iloc[i]['State_j']] ].drop(tmp_yr.iloc[i]['State_i'])
        tmp_SADC_ij[t*2550 + i] = tmp_yr.query("State_i in @intervening_states").groupby('State_i').ADC_i.max().sum()

df['SADC_ij'] = np.repeat(tmp_SADC_ij, df.path_ind.nunique())

In [4]:
# resplit data

df_train = df.query("Year not in @test_years")
df_test = df.query("Year in @test_years")

## XGBoost

In [ ]:
num_folds = 5

param_grid = {'max_depth': np.arange(2,11),
              'n_estimators': np.arange(50,351),
              'learning_rate': np.linspace(0.01,0.5,100),
            }

cpc_xgb , cpcd_xgb, mae_xgb , r_squared_xgb = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
xgb_imprtance = np.empty((path_count, len(df.loc[:,'P_i':].columns)))
for path in df.path_ind.unique()[:path_count]:

    X_train = df_train.query("path_ind == @path").loc[:,'P_i':]

    tmp = np.log(1 + X_train)
    X_train = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_train = np.log(1 + df_train.query("path_ind == @path").M_ij)

    year_ind = df_train.query("path_ind == @path").index
    cv_iterable = []
    for i in range(num_folds):
        cv_iterable.append((np.arange(len(year_ind[year_ind < years[-4-i]])), np.arange(len(year_ind[year_ind == years[-4-i]]))))

    xgbr_fit = RandomizedSearchCV(XGBRegressor(random_state=0), 
                            param_grid, n_iter=50, n_jobs=-1, cv=cv_iterable, verbose=1, error_score='raise').fit(X_train, y_train)
    print(xgbr_fit.best_params_)

    X_test = df_test.query("path_ind == @path").loc[:,'P_i':]
    tmp = np.log(1 + X_test)
    X_test = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_test = df_test.query("path_ind == @path").M_ij

    pred = np.exp(xgbr_fit.predict(X_test)) - 1

    xgb_imprtance[path] = xgbr_fit.best_estimator_.feature_importances_

    cpc_xgb[path] = cpc(y_test, pred)
    cpcd_xgb[path] = cpcd(y_test, pred, df_test.query("path_ind == @path").D_ij)
    mae_xgb[path] = mae(y_test, pred)
    r_squared_xgb[path] = r_squared(y_test, pred)

print("MAE:",       mae_xgb.mean()      , "+\-", norm.ppf(.975) * mae_xgb.std(ddof=1) / np.sqrt(path_count)  )
print("R-squared:", r_squared_xgb.mean(), "+\-", norm.ppf(.975) * r_squared_xgb.std(ddof=1) / np.sqrt(path_count)  )
print("CPC:",       cpc_xgb.mean()      , "+\-", norm.ppf(.975) * cpc_xgb.std(ddof=1) / np.sqrt(path_count)  )
print("CPCD:",      cpcd_xgb.mean()     , "+\-", norm.ppf(.975) * cpcd_xgb.std(ddof=1) / np.sqrt(path_count)  )

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 350, 'max_depth': 9, 'learning_rate': 0.4406060606060606}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 344, 'max_depth': 10, 'learning_rate': 0.401010101010101}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 150, 'max_depth': 10, 'learning_rate': 0.4208080808080808}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 325, 'max_depth': 8, 'learning_rate': 0.4505050505050505}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 307, 'max_depth': 9, 'learning_rate': 0.3564646464646465}
MAE: 1496.6871125817156 +\- 120.35240848724624
R-squared: 0.5984401924021066 +\- 0.12086930700298895
CPC: 0.7353454995765327 +\- 0.016824506197862143
CPCD: 0.9527499753624932 +\- 0.016245557064225244


## ANN

In [ ]:
num_folds = 5

param_grid = {'hidden_layer_sizes': [[width]*depth for depth in np.arange(1,9) for width in np.arange(16,129)],
              'batch_size': [2**4,2**5,2**6,2**7,2**8],
            }

cpc_ann , cpcd_ann, mae_ann , r_squared_ann = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
for path in df.path_ind.unique()[:path_count]:
    X_train = df_train.query("path_ind == @path").loc[:,'P_i':]

    tmp = np.log(1 + X_train)
    X_train = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_train = np.log(1 + df_train.query("path_ind == @path").M_ij)

    year_ind = df_train.query("path_ind == @path").index
    cv_iterable = []
    for i in range(num_folds):
        cv_iterable.append((np.arange(len(year_ind[year_ind < years[-4-i]])), np.arange(len(year_ind[year_ind == years[-4-i]]))))

    ann_fit = RandomizedSearchCV(MLPRegressor(random_state=0, max_iter=1000, early_stopping=True), 
                            param_grid, n_iter=50, n_jobs=-1, cv=cv_iterable, verbose=1, error_score='raise').fit(X_train.values, y_train.values)
    print(ann_fit.best_params_)
    tmp = np.log(1 + df_test.query("path_ind == @path").loc[:,'P_i':])
    X_test = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_test = df_test.query("path_ind == @path").M_ij
    pred = np.exp(ann_fit.predict(X_test.values)) - 1

    cpc_ann[path] = cpc(y_test, pred) 
    cpcd_ann[path] = cpcd(y_test, pred, df_test.query("path_ind == @path").D_ij) 
    mae_ann[path] = mae(y_test, pred)
    r_squared_ann[path] = r_squared(y_test, pred)

print("MAE:", mae_ann.mean()            , "+/-", norm.ppf(.975) * mae_ann.std(ddof=1) / np.sqrt(path_count)      )
print("R-squared:", r_squared_ann.mean(), "+/-", norm.ppf(.975) * r_squared_ann.std(ddof=1) / np.sqrt(path_count)      )
print("CPC:", cpc_ann.mean()             , "+/-", norm.ppf(.975) * cpc_ann.std(ddof=1) / np.sqrt(path_count)      )
print("CPCD:", cpcd_ann.mean()           , "+/-", norm.ppf(.975) * cpcd_ann.std(ddof=1) / np.sqrt(path_count)      )

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [109, 109, 109, 109, 109, 109, 109, 109], 'batch_size': 32}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [97, 97, 97, 97, 97, 97, 97, 97], 'batch_size': 32}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [121, 121, 121, 121, 121], 'batch_size': 32}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [113, 113, 113, 113, 113, 113, 113], 'batch_size': 16}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [105, 105, 105, 105, 105, 105, 105, 105], 'batch_size': 64}
MAE: 1606.885405858689 +/- 112.96054302996139
R-squared: 0.5237449258892373 +/- 0.13048460797056122
CPC: 0.6960933884408619 +/- 0.0125466004745682
CPCD: 0.8776201231781581 +/- 0.06600419048610828


## XGBoost (few features)

In [3]:
num_folds = 5

param_grid = {'max_depth': np.arange(2,11),
              'n_estimators': np.arange(50,351),
              'learning_rate': np.linspace(0.01,0.5,100),
            }

cpc_xgb , cpcd_xgb, mae_xgb , r_squared_xgb = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
# xgb_imprtance = np.empty((path_count, len(df.loc[:,'P_i':].columns)))
for path in df.path_ind.unique()[:path_count]:

    df_train_filtered = df_train.query("path_ind == @path & M_ij != 0")
    X_train = df_train_filtered.loc[:,['P_i','P_j','SP_ij','D_ij']]
    X_train['P_i + SP_ij'] = X_train.P_i + X_train.SP_ij
    X_train['P_i + P_j + SP_ij'] = X_train.P_i + X_train.P_j + X_train.SP_ij
    X_train.drop('SP_ij',axis=1,inplace=True)
    tmp = np.log(X_train)
    X_train = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_train = np.log(df_train_filtered.M_ij)

    year_ind = df_train_filtered.index
    cv_iterable = []
    for i in range(num_folds):
        cv_iterable.append((np.arange(len(year_ind[year_ind < years[-4-i]])), np.arange(len(year_ind[year_ind == years[-4-i]]))))

    xgbr_fit = RandomizedSearchCV(XGBRegressor(random_state=0), 
                            param_grid, n_iter=50, n_jobs=-1, cv=cv_iterable, verbose=1, error_score='raise').fit(X_train, y_train)
    print(xgbr_fit.best_params_)

    df_test_filtered = df_test.query("path_ind == @path & M_ij != 0")
    X_test = df_test_filtered.loc[:,['P_i','P_j','SP_ij','D_ij']]
    X_test['P_i + SP_ij'] = X_test.P_i + X_test.SP_ij
    X_test['P_i + P_j + SP_ij'] = X_test.P_i + X_test.P_j + X_test.SP_ij
    X_test.drop('SP_ij',axis=1,inplace=True)
    tmp = np.log(X_test)
    X_test = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_test = df_test_filtered.M_ij

    pred = np.exp(xgbr_fit.predict(X_test))

    # xgb_imprtance[path] = xgbr_fit.best_estimator_.feature_importances_

    cpc_xgb[path] = cpc(y_test, pred)
    cpcd_xgb[path] = cpcd(y_test, pred, df_test_filtered.D_ij)
    mae_xgb[path] = mae(y_test, pred)
    r_squared_xgb[path] = r_squared(y_test, pred)

print("MAE:",       mae_xgb.mean()      , "+\-", norm.ppf(.975) * mae_xgb.std(ddof=1) / np.sqrt(path_count)  )
print("R-squared:", r_squared_xgb.mean(), "+\-", norm.ppf(.975) * r_squared_xgb.std(ddof=1) / np.sqrt(path_count)  )
print("CPC:",       cpc_xgb.mean()      , "+\-", norm.ppf(.975) * cpc_xgb.std(ddof=1) / np.sqrt(path_count)  )
print("CPCD:",      cpcd_xgb.mean()     , "+\-", norm.ppf(.975) * cpcd_xgb.std(ddof=1) / np.sqrt(path_count)  )

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 339, 'max_depth': 8, 'learning_rate': 0.4851515151515151}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 213, 'max_depth': 10, 'learning_rate': 0.4703030303030303}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 238, 'max_depth': 10, 'learning_rate': 0.45545454545454545}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 249, 'max_depth': 10, 'learning_rate': 0.2871717171717172}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 345, 'max_depth': 10, 'learning_rate': 0.4109090909090909}
MAE: 1306.394335139646 +\- 22.977418410394158
R-squared: 0.7668054724317354 +\- 0.01614941126084192
CPC: 0.785324703187509 +\- 0.003374538186493278
CPCD: 0.9428005650936951 +\- 0.0022889542228997175


## ANN (few features)

In [4]:
num_folds = 5

param_grid = {'hidden_layer_sizes': [[width]*depth for depth in np.arange(1,9) for width in np.arange(16,129)],
              'batch_size': [2**4,2**5,2**6,2**7,2**8],
            }

cpc_ann , cpcd_ann, mae_ann , r_squared_ann = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
for path in df.path_ind.unique()[:path_count]:
    
    df_train_filtered = df_train.query("path_ind == @path & M_ij != 0")
    X_train = df_train_filtered.loc[:,['P_i','P_j','SP_ij','D_ij']]
    X_train['P_i + SP_ij'] = X_train.P_i + X_train.SP_ij
    X_train['P_i + P_j + SP_ij'] = X_train.P_i + X_train.P_j + X_train.SP_ij
    X_train.drop('SP_ij',axis=1,inplace=True)

    tmp = np.log(X_train)
    X_train = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_train = np.log(df_train_filtered.M_ij)

    year_ind = df_train_filtered.index
    cv_iterable = []
    for i in range(num_folds):
        cv_iterable.append((np.arange(len(year_ind[year_ind < years[-4-i]])), np.arange(len(year_ind[year_ind == years[-4-i]]))))

    ann_fit = RandomizedSearchCV(MLPRegressor(random_state=0, max_iter=1000, early_stopping=True), 
                            param_grid, n_iter=50, n_jobs=-1, cv=cv_iterable, verbose=1, error_score='raise').fit(X_train.values, y_train.values)
    print(ann_fit.best_params_)

    df_test_filtered = df_test.query("path_ind == @path & M_ij != 0")
    X_test = df_test_filtered.loc[:,['P_i','P_j','SP_ij','D_ij']]
    X_test['P_i + SP_ij'] = X_test.P_i + X_test.SP_ij
    X_test['P_i + P_j + SP_ij'] = X_test.P_i + X_test.P_j + X_test.SP_ij
    X_test.drop('SP_ij',axis=1,inplace=True)
    tmp = np.log(X_test)
    X_test = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))
    y_test = df_test_filtered.M_ij
    pred = np.exp(ann_fit.predict(X_test.values))

    cpc_ann[path] = cpc(y_test, pred) 
    cpcd_ann[path] = cpcd(y_test, pred, df_test_filtered.D_ij) 
    mae_ann[path] = mae(y_test, pred)
    r_squared_ann[path] = r_squared(y_test, pred)

print("MAE:", mae_ann.mean()            , "+/-", norm.ppf(.975) * mae_ann.std(ddof=1) / np.sqrt(path_count)      )
print("R-squared:", r_squared_ann.mean(), "+/-", norm.ppf(.975) * r_squared_ann.std(ddof=1) / np.sqrt(path_count)      )
print("CPC:", cpc_ann.mean()             , "+/-", norm.ppf(.975) * cpc_ann.std(ddof=1) / np.sqrt(path_count)      )
print("CPCD:", cpcd_ann.mean()           , "+/-", norm.ppf(.975) * cpcd_ann.std(ddof=1) / np.sqrt(path_count)      )

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [125, 125, 125, 125, 125, 125, 125], 'batch_size': 32}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [60, 60, 60, 60, 60, 60, 60, 60], 'batch_size': 64}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [95, 95, 95, 95, 95, 95, 95, 95], 'batch_size': 64}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [117, 117, 117, 117, 117, 117, 117, 117], 'batch_size': 64}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'hidden_layer_sizes': [60, 60, 60, 60, 60, 60, 60], 'batch_size': 32}
MAE: 1436.3460506477875 +/- 44.27761734919961
R-squared: 0.6765473096632345 +/- 0.03176840340076811
CPC: 0.7482679389086287 +/- 0.014059675208968971
CPCD: 0.8732492915742336 +/- 0.033111976873534336
